In [2]:
import pandas as pd
from datasets import load_dataset, Dataset
import os
import re
import torch
from sklearn.model_selection import train_test_split


In [3]:
#setto il device da usare
torch.device("cpu")

device(type='cpu')

In [6]:
#Divido in training e test set
ds_df = pd.read_csv("data/data_csv.csv")
dataset = Dataset.from_pandas(ds_df)
split_set = dataset.train_test_split(test_size=0.1)

train_ds = split_set["train"]
test_ds = split_set["test"]

### TOKENIZATION

In questa sezione si importa il tokenizzatore col quale si tokenizza ciascuna frase nel formato necessario per Bert, alla fine si otterrà un dataset nel formato corretto con tutte le features necessarie per il training

In [8]:
import tokenizers
import transformers
from transformers import BertTokenizer

In [9]:
#si importa il tokenizzatore già configurato (in questo caso: bert-base-italian-cased)
tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-italian-cased")


In [10]:
#facciamo l'encoding di tutto il dataset tokenizzando frase per frase
def encode(sample):
    return tokenizer(sample["text"], padding=True, truncation=True, max_length=512, return_special_tokens_mask=True)

train_set = train_ds.map(encode, batched=True)
test_set = test_ds.map(encode, batched=True)
train_set.set_format('torch', columns=["input_ids", "attention_mask", "token_type_ids"])
test_set.set_format('torch', columns=["input_ids", "attention_mask", "token_type_ids"])


Map:   0%|          | 0/194 [00:00<?, ? examples/s]

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

In [11]:
train_set

Dataset({
    features: ['id', 'text', 'gulp_index', 'input_ids', 'token_type_ids', 'special_tokens_mask', 'attention_mask'],
    num_rows: 194
})

In [12]:
test_set

Dataset({
    features: ['id', 'text', 'gulp_index', 'input_ids', 'token_type_ids', 'special_tokens_mask', 'attention_mask'],
    num_rows: 22
})

### TRAINING DI BERT

Si procede al training di Bert. Il modello dovrà partire da uno stato iniziale con pesi random, per questo non si importa il modello già addestrato, ma si configura semplicemente l'architettura la sua architettura per poi addestrarlo da zero. Si definisce poi una strategia di training e i suoi argomenti per poi addestrare il modello sul trask di Language Modeling. 

In [13]:
from transformers import Trainer, TrainingArguments, TrainerCallback, BertConfig, BertForMaskedLM, DataCollatorForLanguageModeling, set_seed


In [14]:
model_name = "prajjwal1/bert-mini"
model_config = BertConfig.from_pretrained(model_name)

print(model_config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 4,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [15]:
model = BertForMaskedLM(model_config)
model.resize_token_embeddings(len(tokenizer))


Embedding(31102, 256)

In [16]:
model.config

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 4,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31102
}

In [17]:
#usiamo il datacollator per fare le batch per il training
datacollator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.2, return_tensors="pt")

In [18]:
datacollator

DataCollatorForLanguageModeling(tokenizer=PreTrainedTokenizer(name_or_path='dbmdz/bert-base-italian-cased', vocab_size=31102, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}), mlm=True, mlm_probability=0.2, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt')

In [19]:
print(f"Lunghezza del dataset: {len(train_set)}")

Lunghezza del dataset: 194


In [20]:
#definisco una funzione di callback per verificare l'ordinamento dei dati per ogni epoca
class check_ds_order(TrainerCallback):
    def on_epoch_begin(self, args, state, control, train_dataloader, **kwargs):
        f = open(f"train_check.txt", "a")
        f.write(f"\n------------------------ ORDINE DEI DATI ALL'INIZI DELL'EPOCA {int(state.epoch+1)} ------------------------")
        f.write(str(train_dataloader.dataset["input_ids"][:5]))
        f.write("-----------------------------------------------------------------------------------")


In [21]:
#argomenti provvisori, da definire meglio
training_args = TrainingArguments(
    output_dir = "my_pretrained_model",
    evaluation_strategy="steps",
    overwrite_output_dir=True,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    seed=42, 
    )

In [22]:
set_seed(training_args.seed)

In [23]:
open('train_check.txt', 'w').close()

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=datacollator,
    train_dataset=train_set,
    eval_dataset=test_set,
    callbacks=[check_ds_order], 
    )

In [25]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: text, gulp_index, special_tokens_mask, id. If text, gulp_index, special_tokens_mask, id are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
c:\Users\bergo\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 194
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 21


  0%|          | 0/21 [00:00<?, ?it/s]

The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: text, gulp_index, special_tokens_mask, id. If text, gulp_index, special_tokens_mask, id are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 22
  Batch size = 64


{'loss': 10.3255, 'learning_rate': 2.6190476190476192e-05, 'epoch': 1.43}


  0%|          | 0/1 [00:00<?, ?it/s]

...

Saving model checkpoint to my_pretrained_model\checkpoint-20
Configuration saved in my_pretrained_model\checkpoint-20\config.json
Model weights saved in my_pretrained_model\checkpoint-20\pytorch_model.bin


{'eval_loss': 10.123820304870605, 'eval_runtime': 0.6425, 'eval_samples_per_second': 34.243, 'eval_steps_per_second': 1.557, 'epoch': 2.86}




Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from my_pretrained_model\checkpoint-20 (score: 10.123820304870605).


{'train_runtime': 79.2615, 'train_samples_per_second': 7.343, 'train_steps_per_second': 0.265, 'train_loss': 10.256673540387835, 'epoch': 3.0}


TrainOutput(global_step=21, training_loss=10.256673540387835, metrics={'train_runtime': 79.2615, 'train_samples_per_second': 7.343, 'train_steps_per_second': 0.265, 'train_loss': 10.256673540387835, 'epoch': 3.0})